# Filter data based on keywords

## Introduction
The aim of this notebook is to read a set of keywords and a set of scraped data and filter out all non-Covid-19 related entries/rows.

## Import libraries and set up defaults

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
#%xmode Verbose
# Set global default figure size
plt.rc('figure', figsize=(20, 12)) # It's nice with figures that fill the whole space in width
# Show maximum of 8 rows when printing dataframes
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 8
# Show only 4 digits when printing floating point number
np.set_printoptions(precision=4, suppress=True)

## Two sets of keywords

### Key words from [Zhang Lab](https://zhanglab.ccmb.med.umich.edu/COVID-19/) 

#### Read in the keywords

In [2]:
path = "data/01_raw/Covid-19_keywords/"
key_words_df = pd.read_csv(path + "SARS-COV-2_Protein_Names.tsv",
                           sep = '\t',
                           header = 0,
                           usecols = ['ORF','Gene','Gene2','Full_Name'] # Dropping: 'Function', Source and a url
                          )
key_words_df

,ORF,Gene,Gene2,Full_Name
0,ORF1AB,nsp1,NaN,Host translation inhibitor nsp1
1,ORF1AB,nsp2,NaN,Non-structural protein 2
2,ORF1AB,nps3,NaN,Papain-like proteinase
3,ORF1AB,nps4,NaN,Non-structural protein 4
...,...,...,...,...
20,ORF7A,NaN,NaN,Protein 7a
21,ORF8,NaN,NaN,Protein 8
22,N,NaN,NaN,Nucleoprotein
23,ORF10,NaN,NaN,3` UTR


#### Create a unique Python list of keywords

In [3]:
first_term = (key_words_df['ORF']
              .dropna() # Drop np.nan:s
              .unique() # Filter all non-unique values
              .tolist() # Make a python list
             ) + \
key_words_df['Gene'].dropna().unique().tolist() + \
key_words_df['Gene2'].dropna().unique().tolist() + \
key_words_df['Full_Name'].dropna().unique().tolist()
print(first_term)

['ORF1AB', 'S', 'ORF3A', 'E', 'M', 'ORF6', 'ORF7A', 'ORF8', 'N', 'ORF10', 'nsp1', 'nsp2', 'nps3', 'nps4', 'nsp5', 'nsp6', 'nsp7', 'nsp8', 'nsp9', 'nsp10', 'RDRP', 'Hel', 'Exon', 'NendoU', "2'-O-MT", 'Spike', '3CL-PRO', 'Spike trimeric complex (S1, S2, S`)', 'Host translation inhibitor nsp1', 'Non-structural protein 2', 'Papain-like proteinase', 'Non-structural protein 4', '3C-like proteinase', 'Non-structural protein 6', 'Non-structural protein 7', 'Non-structural protein 8', 'Non-structural protein 9', 'Non-structural protein 10', 'RNA-Directed RNA Polymerase', 'Helicase', 'Proofreading exoribonuclease (Guanine-N7 methyltransferase)', 'Uridylate-specific endoribonuclease', "2'-O-methyltransferase", 'Spike surface glycoprotein (monomer)', 'Protein 3a', 'Envelope small membrane proteins', 'Membrane protein', 'Protein 6', 'Protein 7a', 'Protein 8', 'Nucleoprotein', '3` UTR']


### Key words for filtering scraped data from [Mendeley Database](https://data.mendeley.com/)

In [4]:
first_term = ["sars-cov-2", 
              "sars-cov2", 
              "covid19", 
              "covid-19", 
              "sars", 
              "coronavirus", 
              "ncov2019", 
              "ncov-2019",
              "nCov2019",
              "nCov-2019", 
              "COVID2019"]
second_term = ["molecular dynamics", 
               "trajectories", 
               "dynamics simulation", 
               "trajectory", 
               "molecular docking"]

## Read in Scraped data

In [5]:
sc_path = "data/01_raw/Scraped/"
sc_filename = "subset_Secondary_data_extracted_figshare_all_formats.csv"
sc_data = pd.read_csv(sc_path + sc_filename,
                      sep = ",",
                      header = 0,
                      index_col = 0
                     )
sc_data.head()

,Title,Authors,Categories,Keywords,Description,Type of (possible) format,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,S6 molecular dynamics simulation trajectories ...,"['by Marracino, Paolo Havelka, Daniel Průša, J...","['Microbiology', 'Molecular Biology', 'Neurosc...",['1604 Molecular dynamics simulation trajector...,Molecular dynamics simulation trajectories of ...,.xtc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Portable compressed binary MD trajectory file ...,"['by Alessandro Pandini Shahid Khan', 'Alessan...","['10.1371/journal.pcbi.1006796', 'Computationa...","['GROMACS', 'XTC trajectory']",XTC trajectory,.xtc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,S6 molecular dynamics simulation trajectories ...,"['by Marracino, Paolo Havelka, Daniel Průša, J...","['Physical Sciences not elsewhere classified',...",['1604 Molecular dynamics simulation trajector...,Molecular dynamics simulation trajectories of ...,.xtc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Portable compressed binary tCONCOORD trajector...,"['by Alessandro Pandini Shahid Khan', 'Alessan...","['10.1371/journal.pcbi.1006796', 'Computationa...","['tCONCOORD', 'XTC trajectory']",XTC trajectory,.xtc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Portable compressed binary MD trajectory file ...,"['by Alessandro Pandini Shahid Khan', 'Alessan...","['10.1371/journal.pcbi.1006796', 'Computationa...","['GROMACS', 'XTC trajectory']",XTC trajectory,.xtc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Check that the data

In [6]:
sc_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 545 entries, 0 to 544
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Title                      545 non-null    object
 1   Authors                    545 non-null    object
 2   Categories                 545 non-null    object
 3   Keywords                   545 non-null    object
 4   Description                545 non-null    object
 5   Type of (possible) format  545 non-null    object
 6   Unnamed: 7                 2 non-null      object
 7   Unnamed: 8                 2 non-null      object
 8   Unnamed: 9                 2 non-null      object
 9   Unnamed: 10                1 non-null      object
 10  Unnamed: 11                1 non-null      object
 11  Unnamed: 12                1 non-null      object
 12  Unnamed: 13                1 non-null      object
 13  Unnamed: 14                1 non-null      object
 14  Unnamed: 1

There are 1 or 2 rows with almost all `np.nan`:s, let's check out what they contain:

In [7]:
sc_data.describe()

,Title,Authors,Categories,Keywords,Description,Type of (possible) format,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
count,545,545,545,545,545,545,2,2,2,1,1,1,1,1,1
unique,524,325,274,299,397,19,2,2,2,1,1,1,1,1,1
top,more 3d filetypes to test,"['by Kresten Lindorff-Larsen', 'Kresten Lindor...",['Earth and Environmental Sciences'],['PANGAEA'],No description found,.arc,cesium'],TRJ: Tropical japonica in yellow,TEJ: Temperate japonica in blue,ARO-TEJ-TRJ: admixture of ARO with TEJ and TRJ,AUS-IND: admixture of AUS with IND,AUS-TRJ-IND: admixture of AUS with TRJ and IND,TEJ-TRJ: admixture of TRJ with TEJ,TRJ-IND: admixture of TRJ with IND.,.trj
freq,5,40,54,66,77,40,1,1,1,1,1,1,1,1,1


In [8]:
sc_data.loc[sc_data["Unnamed: 7"].notnull()] # https://stackoverflow.com/a/42137824

,Title,Authors,Categories,Keywords,Description,Type of (possible) format,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
243,"Data for ""Measurement of the atom-surface van ...","['by Peyrot, T Šibalić, N. Sortais, Y.R.P. Bro...","['Biochemistry', 'Physical Sciences not elsewh...",['laser spectroscopy,atom-surface interactions,sapphire,cesium'],"The data presented in manuscript ""Measurement ...",.hdf5,NaN,NaN,NaN,NaN,NaN,NaN
461,Structure analysis of USDA rice mini-core coll...,['by Xiaobai Li Wengui Yan Hesham Agrama Limen...,"['Biotechnology', 'Developmental Biology', 'Pl...","['usda', 'mini-core', 'unrooted', 'upmga']",ARO: aromatic in red,AUS: aus in green,IND: Indica in purple,TRJ: Tropical japonica in yellow,TEJ: Temperate japonica in blue,ARO-TEJ-TRJ: admixture of ARO with TEJ and TRJ,AUS-IND: admixture of AUS with IND,AUS-TRJ-IND: admixture of AUS with TRJ and IND,TEJ-TRJ: admixture of TRJ with TEJ,TRJ-IND: admixture of TRJ with IND.,.trj


In [9]:
sc_data.loc[sc_data["Unnamed: 8"].notnull()]

,Title,Authors,Categories,Keywords,Description,Type of (possible) format,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
243,"Data for ""Measurement of the atom-surface van ...","['by Peyrot, T Šibalić, N. Sortais, Y.R.P. Bro...","['Biochemistry', 'Physical Sciences not elsewh...",['laser spectroscopy,atom-surface interactions,sapphire,cesium'],"The data presented in manuscript ""Measurement ...",.hdf5,NaN,NaN,NaN,NaN,NaN,NaN
461,Structure analysis of USDA rice mini-core coll...,['by Xiaobai Li Wengui Yan Hesham Agrama Limen...,"['Biotechnology', 'Developmental Biology', 'Pl...","['usda', 'mini-core', 'unrooted', 'upmga']",ARO: aromatic in red,AUS: aus in green,IND: Indica in purple,TRJ: Tropical japonica in yellow,TEJ: Temperate japonica in blue,ARO-TEJ-TRJ: admixture of ARO with TEJ and TRJ,AUS-IND: admixture of AUS with IND,AUS-TRJ-IND: admixture of AUS with TRJ and IND,TEJ-TRJ: admixture of TRJ with TEJ,TRJ-IND: admixture of TRJ with IND.,.trj


In [10]:
sc_data.loc[sc_data["Unnamed: 9"].notnull()]

,Title,Authors,Categories,Keywords,Description,Type of (possible) format,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
243,"Data for ""Measurement of the atom-surface van ...","['by Peyrot, T Šibalić, N. Sortais, Y.R.P. Bro...","['Biochemistry', 'Physical Sciences not elsewh...",['laser spectroscopy,atom-surface interactions,sapphire,cesium'],"The data presented in manuscript ""Measurement ...",.hdf5,NaN,NaN,NaN,NaN,NaN,NaN
461,Structure analysis of USDA rice mini-core coll...,['by Xiaobai Li Wengui Yan Hesham Agrama Limen...,"['Biotechnology', 'Developmental Biology', 'Pl...","['usda', 'mini-core', 'unrooted', 'upmga']",ARO: aromatic in red,AUS: aus in green,IND: Indica in purple,TRJ: Tropical japonica in yellow,TEJ: Temperate japonica in blue,ARO-TEJ-TRJ: admixture of ARO with TEJ and TRJ,AUS-IND: admixture of AUS with IND,AUS-TRJ-IND: admixture of AUS with TRJ and IND,TEJ-TRJ: admixture of TRJ with TEJ,TRJ-IND: admixture of TRJ with IND.,.trj


In [11]:
sc_data.loc[sc_data["Unnamed: 10"].notnull()]

,Title,Authors,Categories,Keywords,Description,Type of (possible) format,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
461,Structure analysis of USDA rice mini-core coll...,['by Xiaobai Li Wengui Yan Hesham Agrama Limen...,"['Biotechnology', 'Developmental Biology', 'Pl...","['usda', 'mini-core', 'unrooted', 'upmga']",ARO: aromatic in red,AUS: aus in green,IND: Indica in purple,TRJ: Tropical japonica in yellow,TEJ: Temperate japonica in blue,ARO-TEJ-TRJ: admixture of ARO with TEJ and TRJ,AUS-IND: admixture of AUS with IND,AUS-TRJ-IND: admixture of AUS with TRJ and IND,TEJ-TRJ: admixture of TRJ with TEJ,TRJ-IND: admixture of TRJ with IND.,.trj


In [12]:
sc_data.loc[sc_data["Unnamed: 11"].notnull()]

,Title,Authors,Categories,Keywords,Description,Type of (possible) format,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
461,Structure analysis of USDA rice mini-core coll...,['by Xiaobai Li Wengui Yan Hesham Agrama Limen...,"['Biotechnology', 'Developmental Biology', 'Pl...","['usda', 'mini-core', 'unrooted', 'upmga']",ARO: aromatic in red,AUS: aus in green,IND: Indica in purple,TRJ: Tropical japonica in yellow,TEJ: Temperate japonica in blue,ARO-TEJ-TRJ: admixture of ARO with TEJ and TRJ,AUS-IND: admixture of AUS with IND,AUS-TRJ-IND: admixture of AUS with TRJ and IND,TEJ-TRJ: admixture of TRJ with TEJ,TRJ-IND: admixture of TRJ with IND.,.trj


In [13]:
sc_data.loc[sc_data["Unnamed: 12"].notnull()]

,Title,Authors,Categories,Keywords,Description,Type of (possible) format,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
461,Structure analysis of USDA rice mini-core coll...,['by Xiaobai Li Wengui Yan Hesham Agrama Limen...,"['Biotechnology', 'Developmental Biology', 'Pl...","['usda', 'mini-core', 'unrooted', 'upmga']",ARO: aromatic in red,AUS: aus in green,IND: Indica in purple,TRJ: Tropical japonica in yellow,TEJ: Temperate japonica in blue,ARO-TEJ-TRJ: admixture of ARO with TEJ and TRJ,AUS-IND: admixture of AUS with IND,AUS-TRJ-IND: admixture of AUS with TRJ and IND,TEJ-TRJ: admixture of TRJ with TEJ,TRJ-IND: admixture of TRJ with IND.,.trj


In [14]:
sc_data.loc[sc_data["Unnamed: 13"].notnull()]

,Title,Authors,Categories,Keywords,Description,Type of (possible) format,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
461,Structure analysis of USDA rice mini-core coll...,['by Xiaobai Li Wengui Yan Hesham Agrama Limen...,"['Biotechnology', 'Developmental Biology', 'Pl...","['usda', 'mini-core', 'unrooted', 'upmga']",ARO: aromatic in red,AUS: aus in green,IND: Indica in purple,TRJ: Tropical japonica in yellow,TEJ: Temperate japonica in blue,ARO-TEJ-TRJ: admixture of ARO with TEJ and TRJ,AUS-IND: admixture of AUS with IND,AUS-TRJ-IND: admixture of AUS with TRJ and IND,TEJ-TRJ: admixture of TRJ with TEJ,TRJ-IND: admixture of TRJ with IND.,.trj


It appears that both of the rows aren't about covid-19 so let's drop these rows:

In [15]:
sc_clean = sc_data.copy()
sc_clean = sc_data[sc_data["Unnamed: 7"].isnull()]

In [16]:
sc_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 543 entries, 0 to 544
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Title                      543 non-null    object
 1   Authors                    543 non-null    object
 2   Categories                 543 non-null    object
 3   Keywords                   543 non-null    object
 4   Description                543 non-null    object
 5   Type of (possible) format  543 non-null    object
 6   Unnamed: 7                 0 non-null      object
 7   Unnamed: 8                 0 non-null      object
 8   Unnamed: 9                 0 non-null      object
 9   Unnamed: 10                0 non-null      object
 10  Unnamed: 11                0 non-null      object
 11  Unnamed: 12                0 non-null      object
 12  Unnamed: 13                0 non-null      object
 13  Unnamed: 14                0 non-null      object
 14  Unnamed: 1

Now that we have only nulls in the columns 6->14, let's drop them:

In [17]:
sc_clean.drop(columns = ['Unnamed: 7', 
                         'Unnamed: 8',
                         'Unnamed: 9',
                         'Unnamed: 10',
                         'Unnamed: 11',
                         'Unnamed: 12',
                         'Unnamed: 13',
                         'Unnamed: 14',
                         'Unnamed: 15',
                        ],
              inplace = True
             )
sc_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 543 entries, 0 to 544
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Title                      543 non-null    object
 1   Authors                    543 non-null    object
 2   Categories                 543 non-null    object
 3   Keywords                   543 non-null    object
 4   Description                543 non-null    object
 5   Type of (possible) format  543 non-null    object
dtypes: object(6)
memory usage: 29.7+ KB


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [18]:
sc_clean.head()

,Title,Authors,Categories,Keywords,Description,Type of (possible) format
0,S6 molecular dynamics simulation trajectories ...,"['by Marracino, Paolo Havelka, Daniel Průša, J...","['Microbiology', 'Molecular Biology', 'Neurosc...",['1604 Molecular dynamics simulation trajector...,Molecular dynamics simulation trajectories of ...,.xtc
1,Portable compressed binary MD trajectory file ...,"['by Alessandro Pandini Shahid Khan', 'Alessan...","['10.1371/journal.pcbi.1006796', 'Computationa...","['GROMACS', 'XTC trajectory']",XTC trajectory,.xtc
2,S6 molecular dynamics simulation trajectories ...,"['by Marracino, Paolo Havelka, Daniel Průša, J...","['Physical Sciences not elsewhere classified',...",['1604 Molecular dynamics simulation trajector...,Molecular dynamics simulation trajectories of ...,.xtc
3,Portable compressed binary tCONCOORD trajector...,"['by Alessandro Pandini Shahid Khan', 'Alessan...","['10.1371/journal.pcbi.1006796', 'Computationa...","['tCONCOORD', 'XTC trajectory']",XTC trajectory,.xtc
4,Portable compressed binary MD trajectory file ...,"['by Alessandro Pandini Shahid Khan', 'Alessan...","['10.1371/journal.pcbi.1006796', 'Computationa...","['GROMACS', 'XTC trajectory']",XTC trajectory,.xtc


By looking at excerpt, what could be searched is first to look for `first_term` keywords in all columns but `Authors` and `Type of possible) format` and then look for `second_term` keywords in `Keywords` and `Description` columns to focus in on 

## Initialise a boolen pd.Series
The purpose of pd.Series name `found` is to hold a boolen index rows with matches (if a certain keyword is found in the particular entry).

In [32]:
falses = np.zeros(len(sc_data["Title"]), dtype=bool) # https://stackoverflow.com/a/21174962
found1 = pd.Series(data = falses,
                   dtype = bool)
found2 = pd.Series(data = falses,
                   dtype = bool)

## Find all indexes with a match

### Find matches in the first column

In [33]:
first_column_to_search_in = "Keywords"
for word in first_term:
    # Find out if the current search term can be found in the column
    cur_match = sc_data[first_column_to_search_in].str.contains(word) # https://stackoverflow.com/a/15333283
    # Join the found matches to one Series
    found1 = found1 | cur_match

In [34]:
found1.value_counts()

False    39
True      1
dtype: int64

### Find matches in the second column

In [25]:
second_column_to_search_in = "Description"
for word in second_term:
    # Find out if the current search term can be found in the column
    cur_match = sc_data[second_column_to_search_in].str.contains(word) # https://stackoverflow.com/a/15333283
    # Join the found matches to one Series
    found2 = found2 | cur_match

In [11]:
found2

0    False
1    False
2     True
3     True
dtype: bool

### Join the matches from both columns

In [12]:
joined = found1 & found2
joined

0    False
1    False
2    False
3     True
dtype: bool

## Select only rows with matches in both columns

### When searching only with the first term

In [35]:
sc_data[found1]

,Title,Author,Categories,Keywords,Description
21,1 microsecond simulation of nCoV protease 6LU7...,Lenin Dominguez-Ramirez,['Structural Biology (incl. Macromolecular Mod...,"['nCov2019, nCov-2019, COVID2019, Drug Repurpo...",This protein was simulated without its cocryst...


### When searching with two terms

In [13]:
sc_data[joined]

,col_1,col_2
3,sars-cov-2,molecular dynamics
